# Setup

In [3]:
import os
import sys
from pathlib import Path
import random
import numpy as np
import pickle
from typing import List

# Add the src directory to Python path
olmo_core_path = Path.cwd() / "src"
if olmo_core_path.exists():
    sys.path.insert(0, str(olmo_core_path))

from olmo_core.data import (
    NumpyDataLoaderConfig,
    NumpyDatasetConfig,
    NumpyDatasetType,
    TokenizerConfig,
)
from olmo_core.data.numpy_dataset import (
    VSLCurriculumType,
    VSLCurriculumConfig,
)


In [4]:
# Set your new cache base directory (change this to your preferred location)
cache_base = "/home/joberant/NLP_2425b/yoavbaron"

# Set all relevant Hugging Face cache directories
os.environ["HF_HOME"] = cache_base
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_base, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_base, "datasets")
os.environ["HF_TOKENIZERS_CACHE"] = os.path.join(cache_base, "tokenizers")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from olmo_eval import HFTokenizer
from datasets import load_dataset

/home/joberant/NLP_2425b/yoavbaron/anaconda3/envs/ai2-olmo-2-copy/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# Prepare the dataset and dataloader

In [5]:
tokenizer_config = TokenizerConfig.dolma2()
tokenizer = HFTokenizer(
            tokenizer_config.identifier,
            pad_token_id=tokenizer_config.pad_token_id,
            eos_token_id=tokenizer_config.eos_token_id,
            bos_token_id=tokenizer_config.bos_token_id,
        )

include_instance_metadata = False # Set to true when you want tp retrieve metadata, during training set this to False
work_dir = "/home/joberant/NLP_2425b/shirab6/knowledge-analysis-suite/OLMo-core/hp_final/dataset-cache"

with open(os.path.join(cache_base, "knowledge-analysis-suite/OLMo-core/grouped_dict.pkl"), "rb") as f:
    swapping_dict = pickle.load(f)


In [6]:
dataset_config = NumpyDatasetConfig.glob(
    "/home/morg/students/gottesman3/knowledge-analysis-suite/dolma/python/final_tokenizations_with_offsets/no_special/*.npy",  # can be globs
    name=NumpyDatasetType.kas_vsl,
    max_sequence_length=2048,
    min_sequence_length=64,
    vsl_curriculum=VSLCurriculumConfig(name=VSLCurriculumType.grow_p2, num_cycles=8, balanced=False),
    tokenizer=tokenizer_config,
    work_dir=str(work_dir),
    include_instance_metadata=include_instance_metadata,
    swapping_dict=swapping_dict
)
kas_dataset = dataset_config.build()

Loading metadata:   0%|          | 0/8 [00:00<?, ?it/s]

Loading metadata: 100%|██████████| 8/8 [00:00<00:00, 121.98it/s]


In [7]:
kas_dataset

In [8]:
data_loader_config = NumpyDataLoaderConfig(
    global_batch_size=32768,
    seed=0,
    num_workers=8,
    prefetch_factor = 16,
)

dataloader = data_loader_config.build(kas_dataset)
dataloader.reshuffle(1)

In [ ]:
dataloader[0].keys()

TypeError: 'NumpyVSLDataLoader' object is not subscriptable

In [35]:
for i in dataloader:
    print(i.keys())
    print(len(i['index']), i['index'])
    break

Trying to swap chunk 2465503
Swapping chunk 2465503 with 6401103
Trying to swap chunk 2715621
Trying to swap chunk 564478Swapping chunk 2715621 with 4775495

Swapping chunk 564478 with 7690370
Trying to swap chunk 10046982
Swapping chunk 10046982 with 1142565
Trying to swap chunk 3945203
Swapping chunk 3945203 with 4458969
Trying to swap chunk 5149756
Swapping chunk 5149756 with 7307548
Trying to swap chunk 564233
Swapping chunk 564233 with 1400467
Trying to swap chunk 7442658
Swapping chunk 7442658 with 3266546
dict_keys(['input_ids', 'attention_mask', 'index'])
17 tensor([ 3626703,  7216007,  9127750,  4308706,  3561431,  4620958,  7592063,
        10195124, 10161708,  3590584,  7534629,  3471291,  6353848,  2465503,
         4114366,  5607351,  1918612])


In [25]:
next_item = next(dataloader)

TypeError: 'NumpyVSLDataLoader' object is not an iterator

# Load PopQA dataset and filter entities

In [9]:
def get_important_chunks(dataset, min_num_chunks, max_num_chunks, instance_lengths):
    # Filter the dataset
    filtered_dataset = dataset['train'].filter(
        lambda example: min_num_chunks <= example['subject_num_chunks'] <= max_num_chunks
    )   

    # Create list of dictionaries with subject info and chunk lengths
    result_list = []    

    for example in filtered_dataset:
        
        subject_name = example['subj']
        subject_id = example['subj_id']
        chunks = example['subject_chunks']
        num_chunks = example['subject_num_chunks']

        chunk_lengths = instance_lengths[chunks]

        if subject_name == 'Madison':
            print(chunks)

        # Sort chunks by their lengths (descending order)
        if len(chunk_lengths) > 0:
            # Create pairs of (chunk, length) and sort by length
            chunk_length_pairs = list(zip(chunks, chunk_lengths))
            chunk_length_pairs.sort(key=lambda x: x[1], reverse=True)

            # Separate back into sorted chunks and lengths
            sorted_chunks = [pair[0] for pair in chunk_length_pairs]
            sorted_lengths = [pair[1] for pair in chunk_length_pairs]
        else:
            sorted_chunks = chunks
            sorted_lengths = chunk_lengths

        subject_dict = {
            'entity_id': subject_id,
            'num_chunks': num_chunks,
            'chunks': sorted_chunks,
            'chunks_lengths': sorted_lengths
        }
        
        result_list.append(subject_dict)    

    # Sort the list by number of chunks (descending order)
    result_list.sort(key=lambda x: x['num_chunks'], reverse=True)

    return result_list


In [10]:
ds = load_dataset("dhgottesman/popqa-kas")

"""
importsnt chunks has the following structure:
        {
            'entity_id': subject_id,
            'num_chunks': num_chunks,
            'chunks': sorted_chunks,
            'chunks_lengths': sorted_lengths
        }
"""
important_chunks = get_important_chunks(ds, 50, 100, kas_dataset.get_instance_lengths())

[820777, 941125, 976593, 1132074, 4955122, 5784515, 5893462, 6285853, 6301590, 6569641, 6709763, 982625, 1069094, 1174705, 1175438, 1188820, 3892745, 6043687, 7585232, 2052326, 4449953, 6471684, 6415190, 263910, 430203, 436392, 857271, 964356, 1492452, 1751563, 1953559, 2146414, 2244409, 2359129, 2390139, 2799444, 2839387, 3210372, 3220392, 3475484, 3514601, 3586220, 3735351, 3892746, 3950691, 4321806, 4489791, 4814079, 5279548, 5384395, 5985105, 6007311, 6123047, 6773572, 6837529, 6855838, 7299974, 7309924, 7714134, 8089661, 9309412, 9423790, 9496875, 9866817, 4171091, 8267712, 3130262, 8233434, 1211449, 6231806, 9180749, 342090, 534038, 1647807, 1961324, 2725096, 3113130, 3123297, 3461098, 4020853, 4255747, 4622325, 4713269, 4976107, 6497698, 6498978, 6855755, 7551764, 7688290, 7901132, 9352219, 10391979, 7551765, 7551766, 879397, 3232229, 7994593, 10329160, 7585233]
[500753, 500754, 500756, 7899334, 9766174, 7586543, 3930628, 4385903, 5956585, 5902917, 7143593, 8695947, 1058648, 376

# Load original batches

In [11]:
all_batches = np.load("/home/morg/students/gottesman3/knowledge-analysis-suite/OLMo-core/batch_indices.npy", allow_pickle=True)

# Sample injection points

In [12]:
import random

def sample_injection_points(total_steps, num_points_to_sample, max_num_chunks, interval, seed=None):
    """
    Samples unique injection points from a valid starting range to avoid overflow 
    when assigning chunk indices.

    Args:
        total_steps (int): The maximum possible step value (exclusive upper bound).
        num_points_to_sample (int): Number of injection points to sample.
        max_num_chunks (int): Maximum num_chunks across all entities.
        interval (int): Distance between chunk indices.
        seed (int, optional): Seed for reproducibility.

    Returns:
        List[int]: Sorted list of valid injection starting points.
    """
    if seed is not None:
        random.seed(seed)

    max_valid_start = total_steps - (max_num_chunks - 1) * interval
    if max_valid_start <= 0:
        raise ValueError("Interval and chunk size too large for total steps.")

    if num_points_to_sample > max_valid_start:
        raise ValueError("Cannot sample more injection points than available valid start points.")

    sampled_points = random.sample(range(max_valid_start), k=num_points_to_sample)
    return sorted(sampled_points)


def assign_indices_to_entities(entities, injection_points, interval):
    """
    Assigns indices to each entity starting at a given injection point with spacing.

    Args:
        entities (List[dict]): List of entity dicts.
        injection_points (List[int]): List of sampled injection start points.
        interval (int): Distance between chunk indices.

    Returns:
        Dict[str, List[int]]: Mapping from entity name to list of indices.
    """
    if len(entities) != len(injection_points):
        raise ValueError("Number of entities must match number of injection points.")

    result = {}

    for entity, start in zip(entities, injection_points):
        entity_id = entity['entity_id']
        num_chunks = entity['num_chunks']
        indices = [start + i * interval for i in range(num_chunks)]
        result[entity_id] = indices

    return result



In [13]:
interval = 5

total_number_of_batches = dataloader.total_batches
injection_points = sample_injection_points(total_number_of_batches, len(important_chunks), 100, interval, 0)
all_injection_points_per_entity = assign_indices_to_entities(important_chunks, injection_points, interval)
#all_injection_points_per_entity

# Build Swapping Dictionary

In [14]:
def shloop(
    injection_points: List[int],
    entity_data: dict,
    batch_to_chunks_map: dict,
) -> dict:
    """
    """
    # 1. Get entity chunks available for swapping and their lengths

    ent_chunk_to_len = dict(zip(entity_data['chunks'], entity_data['chunks_lengths']))
    ent_len_to_chunk = {v: k for k, v in ent_chunk_to_len.items()}

    # casting to int but might want to edit this
    batch_id_to_len = {}
    batch_len_to_id = {}
    for batch in injection_points:
        batch_len = int(32768 / len(batch_to_chunks_map[batch]))
        batch_id_to_len[batch] = batch_len
        batch_len_to_id[batch_len] = batch

    # 2. Calculate the injection span
    num_chunks = len(entity_data['chunks'])
    #print(f"Injection span: {list(injection_points)}")
    if len(injection_points) != num_chunks:
        f"Entity {entity_data['entity_id']} expected {num_chunks} injection points, but got {len(injection_points)}."
    

    sb = sorted(batch_len_to_id.keys())   
    se = sorted(ent_len_to_chunk.keys())

    chunks_to_batches = []
    for len_e in se:
        for len_b in sb:
            if len_b == len_e:
                #print(len_e, len_b)
                chunk_id = ent_len_to_chunk[len_e]
                batch_id = batch_len_to_id[len_b]

                #print(f"Chunk {chunk_id} with length {len_e} will be swapped with batch {batch_id} with length {len_b}")
                # get a random chunk id from the batch
                chunk_id_from_batch = random.choice(batch_to_chunks_map[batch_id])

                if [chunk_id, chunk_id_from_batch] in chunks_to_batches or [chunk_id_from_batch, chunk_id] in chunks_to_batches:
                    print(chunk_id, chunk_id_from_batch, "already in")
                
                chunks_to_batches.append([chunk_id, chunk_id_from_batch])
                chunks_to_batches.append([chunk_id_from_batch, chunk_id])
                #chunks_to_batches[chunk_id] = chunk_id_from_batch # chunk e goes to chunk e' in batch b
                #chunks_to_batches[chunk_id_from_batch] = chunk_id # add the symetric mapping

                ent_len_to_chunk.pop(len_e) # pop one of the lengths
                ent_chunk_to_len.pop(chunk_id) # pop the chunk from the entity and pop one of the lengths
                batch_len_to_id.pop(len_b)
                batch_id_to_len.pop(batch_id) # pop the batch and the length from the batch
                break
                
    # ranmly match the rest of the chunks
    for chunk_id, batch_id in zip(ent_chunk_to_len.keys(), batch_id_to_len.keys()):
        if chunk_id not in chunks_to_batches:
            chunk_id_from_batch = random.choice(batch_to_chunks_map[batch_id])

            if [chunk_id, chunk_id_from_batch] in chunks_to_batches or [chunk_id_from_batch, chunk_id] in chunks_to_batches:
                    print(chunk_id, chunk_id_from_batch, "already in")
                
            chunks_to_batches.append([chunk_id, chunk_id_from_batch])
            chunks_to_batches.append([chunk_id_from_batch, chunk_id])
            
            #chunks_to_batches[chunk_id] = chunk_id_from_batch
            #chunks_to_batches[chunk_id_from_batch] = chunk_id

    return chunks_to_batches

In [15]:
full_mapping = []
for i, important_chunk in enumerate(important_chunks):
    pts = all_injection_points_per_entity[important_chunk['entity_id']]

    # The 'important_chunk' variable is the integer you need.
    # Pass it directly to your function.
    res = shloop(
        pts,
        important_chunk,
        all_batches
    )
    # extend full mapping with the result
    full_mapping.extend(res)

In [16]:
combined_length = sum(entity['num_chunks'] for entity in important_chunks)
print("Combined length of all lists under important_chunks:", combined_length)

Combined length of all lists under important_chunks: 72433


In [17]:
len(full_mapping)

144866

In [18]:
grouped_dict = {}
for key, value in full_mapping:
    if key not in grouped_dict:
        grouped_dict[key] = []
    grouped_dict[key].append(value)

In [19]:
with open(os.path.join(cache_base, "knowledge-analysis-suite/OLMo-core/grouped_dict.pkl"), "wb") as f:
    pickle.dump(grouped_dict, f)

In [20]:
from collections import defaultdict

# Dictionary to track occurrences of each item at each position (0 or 1)
position_counts = [defaultdict(int), defaultdict(int)]

for pair in full_mapping:
    position_counts[0][pair[0]] += 1
    position_counts[1][pair[1]] += 1

duplicates = {
    0: [item for item, count in position_counts[0].items() if count > 1],
    1: [item for item, count in position_counts[1].items() if count > 1]
}

print("Items appearing more than once at position 0:", len(duplicates[0]))
print("Items appearing more than once at position 1:", len(duplicates[1]))

Items appearing more than once at position 0: 12873
Items appearing more than once at position 1: 12873


In [21]:
from collections import defaultdict

# Track chunk_id -> list of entity_ids where it appears
chunk_to_entities = defaultdict(set)

for entity in important_chunks:
    entity_id = entity['entity_id']
    for chunk_id in entity['chunks']:
        chunk_to_entities[chunk_id].add(entity_id)

# Find chunk_ids that appear in more than one entity
duplicate_chunks = {chunk_id: list(entity_ids) for chunk_id, entity_ids in chunk_to_entities.items() if len(entity_ids) > 1}

print(f"Number of chunk ids appearing in multiple entities: {len(duplicate_chunks)}")
for chunk_id, entity_ids in list(duplicate_chunks.items())[:100]:  # show first 10 for brevity
    print(f"Chunk ID {chunk_id} appears in entities: {entity_ids}")

Number of chunk ids appearing in multiple entities: 2534
Chunk ID 10380177 appears in entities: [1258065, 2083978]
Chunk ID 2189467 appears in entities: [1258065, 978452]
Chunk ID 3066924 appears in entities: [1258065, 2083978]
Chunk ID 10378761 appears in entities: [1258065, 2083978]
Chunk ID 3277628 appears in entities: [1200906, 22940, 188071]
Chunk ID 3373418 appears in entities: [22940, 188071]
Chunk ID 3373419 appears in entities: [22940, 188071]
Chunk ID 3507554 appears in entities: [2143529, 22940]
Chunk ID 4460895 appears in entities: [22940, 188071]
Chunk ID 9734793 appears in entities: [929961, 22940, 1203333]
Chunk ID 3107368 appears in entities: [22940, 188071]
Chunk ID 1560730 appears in entities: [744404, 22940]
Chunk ID 3550098 appears in entities: [1774154, 22940, 188071]
Chunk ID 4460896 appears in entities: [22940, 188071]
Chunk ID 2661416 appears in entities: [2886218, 22940]
Chunk ID 3395466 appears in entities: [723619, 22940]
Chunk ID 8937698 appears in entities:

In [22]:
total_len = 0
for d in full_mapping:
    total_len += len(d)

total_len / 2

144866.0

In [ ]:
# with open('/home/joberant/NLP_2425b/shirab6/knowledge-analysis-suite/OLMo-core/swapping_dict.pkl', 'wb') as f:
    # pickle.dump(swapping_dict, f)

PermissionError: [Errno 13] Permission denied: '/home/joberant/NLP_2425b/shirab6/knowledge-analysis-suite/OLMo-core/swapping_dict.pkl'

In [ ]:
# To load it back later
with open('/home/joberant/NLP_2425b/shirab6/knowledge-analysis-suite/OLMo-core/swapping_dict.pkl', 'rb') as f:
    swapping_dict = pickle.load(f)

# Rebuild dataset and dataloader with swapped chunk indices

In [ ]:
tokenizer_config = TokenizerConfig.dolma2()
tokenizer = HFTokenizer(
            tokenizer_config.identifier,
            pad_token_id=tokenizer_config.pad_token_id,
            eos_token_id=tokenizer_config.eos_token_id,
            bos_token_id=tokenizer_config.bos_token_id,
        )

include_instance_metadata = False # Set to true when you want tp retrieve metadata, during training set this to False
work_dir = "/home/joberant/NLP_2425b/shirab6/knowledge-analysis-suite/OLMo-core/hp_final/dataset-cache"


dataset_config = NumpyDatasetConfig.glob(
    "/home/morg/students/gottesman3/knowledge-analysis-suite/dolma/python/final_tokenizations_with_offsets/no_special/*.npy",  # can be globs
    name=NumpyDatasetType.kas_vsl,
    max_sequence_length=2048,
    min_sequence_length=64,
    vsl_curriculum=VSLCurriculumConfig(name=VSLCurriculumType.grow_p2, num_cycles=8, balanced=False),
    tokenizer=tokenizer_config,
    work_dir=str(work_dir),
    include_instance_metadata=include_instance_metadata,
    swapping_dict = swapping_dict,
)

reordered_dataset = dataset_config.build()


Loading metadata: 100%|██████████| 8/8 [00:00<00:00, 279.79it/s]


In [ ]:
data_loader_config = NumpyDataLoaderConfig(
    global_batch_size=32768,
    seed=0,
    num_workers=8,
    prefetch_factor = 16,
)

dataloader = data_loader_config.build(reordered_dataset)
dataloader.reshuffle(1)

In [ ]:
sorted_keys = list(swapping_dict.keys())
sorted_keys.sort()